In [181]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBClassifier
from xgboost import plot_importance
import xgboost
import eli5 
from eli5.sklearn import PermutationImportance 
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
import warnings
warnings.filterwarnings('ignore')
from gensim.models import Word2Vec

global PartsData_pd

In [175]:
################################################################################## 테스트 ###############################################################

# 프로젝트 파일 읽기
# 리스트로 변환 append 후 중복 리스트 제거
filepath_base = 'D:/prj_List/'
filealllist =  os.listdir(filepath_base)
prjpath = filealllist[0]

prjpath =  'EBR36371901-BOT.prj'
f = open(filepath_base+prjpath , 'rt', encoding='UTF8')

PatternData_list = []
PartsData_list = []
PatternData = 0
PartsData = 0
for i in f:
    i = i.strip()
    #######################################
    #### PatternData ####
    if PatternData == 1 and i == '':
        PatternData = 2
    if PatternData == 1:
        # print(i)
        i_split = i.split(' ')
        PatternData_list.append(i_split[:6])
    if i.__contains__('[PatternData<') and PatternData == 0:
        PatternData = 1
    #######################################

    #######################################
    #### PartsData ####
    if PartsData == 1 and i == '':
        PartsData = 2
    if PartsData == 1:
        # print(i)
        i_split = i.split(' ')
        PartsData_list.append(i_split[:2])  #PartsData의 값이 빈곳이 있어 배열 제한
    if i.__contains__('[PartsData') and PartsData == 0:
        PartsData = 1
    #######################################
f.close()

#prj에서 읽어온 리스트를 판다스로 변환
#### PatternData ####
PatternData_col = PatternData_list[0]
PatternData_array = np.array(PatternData_list[1:])
PatternData_pd = pd.DataFrame(PatternData_array , columns=PatternData_col)
#### PartsData ####
PartsData_col = PartsData_list[0]
PartsData_array = np.array(PartsData_list[1:])
PartsData_pd = pd.DataFrame(PartsData_array , columns=PartsData_col)

#PartsData 에 PatternData 의 X,Y 값 일치하여 추가
def PartsData_Name(s):
    # print(s)
    k = PartsData_pd[PartsData_pd['IDNUM'] == s]
    N = k.iloc[0]['NAME']
    return N

# PatternData 에 Name 일치하여 추가
PatternData_pd['NAME'] = PatternData_pd['PARTS'].apply(PartsData_Name)

#필요 데이터만 추출
need_col = ['NAME' , 'PARTS' , 'X' , 'Y']
Data_pd = PatternData_pd[need_col]
Data_pd['NAME'] = Data_pd['NAME'].str.replace('"' , '')

# X -> Y 순으로 정렬
Data_pd['X'] = pd.to_numeric(Data_pd['X'])
Data_pd['Y'] = pd.to_numeric(Data_pd['Y'])
Data_pd = Data_pd.sort_values(by = 'X')
Data_pd = Data_pd.sort_values(by = 'Y')
Data_pd = Data_pd.reset_index(drop=True)

Data_pd


,NAME,PARTS,X,Y
0,EBC64038515,14316,-62.70,22.95
1,EBK63351301,14349,-60.70,22.95
2,EAH63174401,15190,-62.20,25.95
3,EBC64038516,14300,-66.10,27.05
4,0CK331BKFDA,15191,-106.80,27.45
...,...,...,...,...
656,EAM62152001,15153,-62.60,177.05
657,EAM62152001,15153,-67.30,177.05
658,EAP61829301,15219,-111.15,177.25
659,EBC62528412,15228,-70.25,177.25


In [174]:
PatternData_pd

,IDNUM,O,X,Y,A,PARTS
0,1,1,-112.400,135.650,90.000,15191
1,2,1,-122.500,119.650,0.000,15191
2,3,1,-125.000,118.450,0.000,14347
3,4,1,-47.700,52.550,0.000,14347
4,5,1,-108.950,176.150,90.000,15285
...,...,...,...,...,...,...
657,658,1,-68.600,155.270,0.000,15373
658,659,1,-76.580,158.450,90.000,15373
659,660,1,-59.900,161.450,90.000,21908
660,661,1,-49.800,134.850,-90.000,15353


In [176]:
#PartsData 에 PatternData 의 X,Y 값 일치하여 추가


def PartsData_Name(s):
    global PartsData_pd
    k = PartsData_pd[PartsData_pd['IDNUM'] == s]
    N = k.iloc[0]['NAME']
    return N

def get_PartXY(filepath_base , prjpath):
    global PartsData_pd
    f = open(filepath_base+prjpath , 'rt', encoding='UTF8')

    PatternData_list = []
    PartsData_list = []
    PatternData = 0
    PartsData = 0
    for i in f:
        i = i.strip()
        #######################################
        #### PatternData ####
        if PatternData == 1 and i == '':
            PatternData = 2
        if PatternData == 1:
            # print(i)
            i_split = i.split(' ')
            PatternData_list.append(i_split[:6])
        if i.__contains__('[PatternData<') and PatternData == 0:
            PatternData = 1
        #######################################

        #######################################
        #### PartsData ####
        if PartsData == 1 and i == '':
            PartsData = 2
        if PartsData == 1:
            # print(i)
            i_split = i.split(' ')
            PartsData_list.append(i_split[:2])  #PartsData의 값이 빈곳이 있어 배열 제한
        if i.__contains__('[PartsData') and PartsData == 0:
            PartsData = 1
        #######################################
    f.close()

    #prj에서 읽어온 리스트를 판다스로 변환
    #### PatternData ####
    PatternData_col = PatternData_list[0]
    PatternData_array = np.array(PatternData_list[1:])
    PatternData_pd = pd.DataFrame(PatternData_array , columns=PatternData_col)
    #### PartsData ####
    PartsData_col = PartsData_list[0]
    PartsData_array = np.array(PartsData_list[1:])
    PartsData_pd = pd.DataFrame(PartsData_array , columns=PartsData_col)

    # PatternData 에 Name 일치하여 추가
    PatternData_pd['NAME'] = PatternData_pd['PARTS'].apply(PartsData_Name)

    #필요 데이터만 추출
    need_col = ['NAME' , 'PARTS' , 'X' , 'Y']
    Data_pd = PatternData_pd[need_col]
    Data_pd['NAME'] = Data_pd['NAME'].str.replace('"' , '')

    # X -> Y 순으로 정렬
    Data_pd['X'] = pd.to_numeric(Data_pd['X'])
    Data_pd['Y'] = pd.to_numeric(Data_pd['Y'])
    Data_pd = Data_pd.sort_values(by = 'X')
    Data_pd = Data_pd.sort_values(by = 'Y')
    Data_pd = Data_pd.reset_index(drop=True)
    
    return Data_pd

In [186]:
# 해당 폴더의 prj 파일 읽어와서 각 모델에 대한 X.Y 좌표 순서대로 부품명을 PartXY_list 에 추가하기

PartXY_list = []

filepath_base = 'D:/prj_List/'
filealllist =  os.listdir(filepath_base)

for i in filealllist:
    # print(i)
    result = get_PartXY(filepath_base , i)
    result = result['NAME']
    result_np = result.to_numpy()
    # print(result_np.shape)
    result_list = result_np.tolist()
    PartXY_list.append(result_list)

print(len(PartXY_list))

with open("PartList_230927.pkl","wb") as f:
    pickle.dump(PartXY_list, f)

126


In [187]:
# 좌표 순서대로 저장된 부품명을 리스트로 불러와서 부품을 3차원으로 벡터화

with open("PartList_230927.pkl","rb") as f:
    sentences = pickle.load(f)

# Word2Vec 모델 학습
model = Word2Vec(sentences, vector_size=3, window=2, min_count=1, workers=4, sg=1)

In [198]:
# 중복 부품 제거 후, 부품에 벡터값이 포함토록 리스트 화
alllist = []
for i in sentences:
    for j in i:
        alllist.append(j)
alllist_set = set(alllist)  #부품 중복 제거

word_vector_list = []
for i in alllist_set:
    word_list = []
    word_list.append(i)
    word_vector = model.wv[i]
    for j in word_vector:
        word_list.append(j)
    word_vector_list.append(word_list)  #부품과 벡테값 리스트에 담기

In [199]:
# 리스트를 넘파이로 변환하고 판다스로 저장

word_vector_list_array = np.array(word_vector_list)
word_vector_list_pd = pd.DataFrame(word_vector_list_array , columns=[['NAME' , 'V1' , 'V2' , 'V3']])
word_vector_list_pd.to_csv("PartVector_230927.csv")
word_vector_list_pd


,NAME,V1,V2,V3
0,EBC62528412,-0.4116163,0.5538005,0.81969345
1,EAN62931901,1.0436801,1.1311411,2.037103
2,EBC62528415,1.4830259,-1.4128082,3.8268225
3,EAP61829301,-0.65744805,-0.35548165,0.9733705
4,0RH0182C622,-0.7301614,3.6443722,2.2038972
...,...,...,...,...
268,0RJ3602C478,-0.4128285,0.4758558,0.89920324
269,EAE64301601,-0.31188744,0.010568643,0.97606367
270,0LCTD00034A,-1.2463061,-1.0323027,1.5240749
271,EAG64914801,1.0450382,1.0118977,1.8729384
